In [ ]:
import os
import cv2
from datetime import date
import json
import itertools

import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel
import utm
import pandas as pd

In [ ]:
geojson_path = "./data/target_zone.geojson"

with open("./secrets/sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"], 
    credentials["password"],
    "https://scihub.copernicus.eu/dhus"
)

shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2020, 7, 27), date(2020, 8, 8)),
    platformname="Sentinel-2",
    processinglevel="Level-2A",
    cloudcoverpercentage=(0, 30)
)

images_df = api.to_dataframe(images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
images_df[key_columns].head()

In [ ]:
uuid = "4172581f-67cf-4c08-9250-be29c86bfae7"
# uuid = "a6889608-c2b4-4b0d-a731-a7f0c2da8951"
# api.download("0f30f930-88c3-4540-b478-414df01ef414")

Dans la cellule suivante, on a :
* la première valeur correspond au 27 juillet 2020 (avant l'incendie)
* la deuxième date du 29 juillet mais l'image n'est pas exploitable (quasiment que du noir)
* la troisième correspond au 6 août 2020 donc après l'incendie

In [ ]:
# article file
# image_folder = "./data/S2B_MSIL2A_20200727T105619_N0214_R094_T30TXQ_20200727T144035.SAFE"
# image_folder = "./data/S2A_MSIL2A_20200729T105031_N0214_R051_T30TXQ_20200729T134025.SAFE"
image_folder = "./data/S2B_MSIL2A_20200806T105619_N0214_R094_T30TXQ_20200806T131026.SAFE"

In [ ]:
def get_band(image_folder, band, resolution=10):
    subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    image_folder_path  = f"{image_folder}/GRANULE/{subfolder}/IMG_DATA/R{resolution}m"
    image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    selected_file = [im for im in image_files if im.split("_")[2] == band][0]
    
    with rasterio.open(f"{image_folder_path}/{selected_file}") as infile:
        img = infile.read()
    
    return img

In [ ]:
band_dict = {
    band: get_band(image_folder, band, resolution=20)
    # for band in ["B02", "B03", "B04", "B08"]
    for band in ["B12", "B8A", "B04"]
}

In [ ]:
# TODO

# NDVI
# NDVI = (B8 - B04) / (B8 + B04)

# (PIR - Red) / (PIR + Red)

# SWIR
# SWIR = (B12, B8A, B04)

# Nuages ? robuste
# Shapefile en sortie

Ici, le NDVI correspon à un rapport de fréquences qui devrait avoir des valeurs entre -1 et 1. Or ce n'est pas le cas, donc il faut vérifier quel est le problème

In [ ]:
# vérifier valeurs entre -1 (eau, nuages...) et 1 (végétation)
# B = B04, G = B03, R = B02
R = band_dict["B04"]
G = band_dict["B03"]
B = band_dict["B02"]
B8 = band_dict["B08"]
NDVI = (B8 - R) / (B8 + R)

NDVI_np = np.array(NDVI)
print(np.max(np.max(NDVI_np, axis=1)))
print(np.min(np.min(NDVI_np, axis=1)))
# filtrer valeurs négatives
# pas de valeurs négatives ? vérifier

In [ ]:
NDVI_processed = NDVI / 8
NDVI_processed = NDVI_processed.astype(int)

plt.figure(figsize=(16, 9))
plt.imshow(NDVI_processed)
plt.show()

In [ ]:
# Recreating an RGB image
# img = cv2.merge((band_dict["B04"], band_dict["B03"], band_dict["B02"])) # RGB
# img = cv2.merge((band_dict["B08"], band_dict["B04"], band_dict["B03"])) # Infrared
img = cv2.merge((band_dict["B12"], band_dict["B8A"], band_dict["B04"])) # SWIR
img = img.squeeze()

plt.figure(figsize=(16, 9))
plt.imshow(img)
plt.show()

In [ ]:
img_processed = img / 8
img_processed = img_processed.astype(int)

plt.figure(figsize=(16, 9))
plt.imshow(img_processed)
plt.show()

Splitting the images to machine learning ready sizes

In [ ]:
frag_count = 18
frag_size = int(img_processed.shape[0] / frag_count)
frag_dict = {}

for y, x in itertools.product(range(frag_count), range(frag_count)):
    frag_dict[(x, y)] = img_processed[y * frag_size: (y + 1) * frag_size,
        x * frag_size: (x + 1) * frag_size, :]

Link data to the satellite images

Par tâtonnements j'ai trouvé la zone de l'incendie

In [ ]:
# for k in range(10, 17):
#     plt.figure(figsize=(16, 9))
#     plt.imshow(frag_dict[(k, 12)])
#     plt.show()

plt.figure(figsize=(16, 9))
plt.imshow(frag_dict[(15, 12)])
plt.show()

In [ ]:
def get_tci_file_path(image_folder):
    subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    image_folder_path  = f"{image_folder}/GRANULE/{subfolder}/IMG_DATA/R10m"
    image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    selected_file = [im for im in image_files if im.split("_")[2] == "TCI"][0]
    
    return f"{image_folder_path}/{selected_file}"

tci_file_path = get_tci_file_path(image_folder)

In [ ]:
# Testing with the center of fragment 10, 10
pixel_column = int(frag_size * 10.5)
pixel_row = int(frag_size * 10.5)
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

UTM: Universal Tranverse Mercator

In [ ]:
transform = rasterio.open(tci_file_path, driver='JP2OpenJPEG').transform
zone_number = int(tci_file_path.split("/")[-1][1:3])
zone_letter = tci_file_path.split("/")[-1][0]

utm_x, utm_y = transform[2], transform[5]

In [ ]:
# Converting pixel position to UTM
east = utm_x + pixel_column * 10
north = utm_y + pixel_row * - 10

# Converting UTM to latitude and longitude
latitude, longitude = utm.to_latlon(east, north, zone_number, zone_letter)

In [ ]:
print(f"latitude, longitude: {latitude}, {longitude} ")

In [ ]:
# Converting latitude and longitude to UTM
east, north, zone_number, zone_letter = utm.from_latlon(latitude, longitude, force_zone_number=zone_number)

# Converting UTM to column and row
pixel_column = round((east - utm_x) / 10)
pixel_row = round((north - utm_y) / - 10)

In [ ]:
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

In [ ]:
plt.figure(figsize=(16, 9))
plt.subplot(221)
plt.imshow(frag_dict[(14, 12)])
plt.subplot(222)
plt.imshow(frag_dict[(15, 12)])
plt.subplot(223)
plt.imshow(frag_dict[(14, 13)])
plt.subplot(224)
plt.imshow(frag_dict[(15, 13)])
plt.savefig("test2.png")
plt.show()

In [ ]:
from PIL import Image

plt.figure(figsize=(16, 9))
plt.subplot(221)
plt.imshow(Image.open("test(20,16).png"))
plt.xticks([]), plt.yticks([])
plt.subplot(222)
plt.imshow(Image.open("test(21,16).png"))
plt.xticks([]), plt.yticks([])
plt.subplot(223)
plt.imshow(Image.open("test(20,17).png"))
plt.xticks([]), plt.yticks([])
plt.subplot(224)
plt.imshow(Image.open("test(21,17).png"))
plt.xticks([]), plt.yticks([])
plt.gca().set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
            hspace = 0, wspace = -0.61)
plt.margins(0, 0)
plt.savefig("test3.png", bbox_inches='tight', pad_inches=0)
plt.show()

Edge detection with Canny

In [ ]:
img = cv2.imread("test3.png")
edges = cv2.Canny(img, 100, 200)

plt.figure(figsize=(16, 9))
plt.subplot(121)
plt.imshow(img)
plt.title('Original Image')
plt.xticks([])
plt.yticks([])
plt.subplot(122)
plt.imshow(edges, cmap='gray')
plt.title('Edge Image')
plt.xticks([])
plt.yticks([])
plt.show()